In [1]:
import numpy as np

from pymatgen import Molecule
from pymatgen.core.operations import SymmOp
from pymatgen.util.coord_utils import get_angle

In [2]:
class Polymer(object):
    #reordered variables start -> middle -> end n_units link_dist
    def __init__(self, start_monomer, s_head, s_tail, monomer, head, tail, end_monomer, e_head, e_tail, n_units, link_distance=1.0):
        """
        Args:
            monomer (Molecule): The monomer
            n_units (int): number of monomer units
            head (int): index of the atom in the monomer that forms the head
            tail (int): tail atom index. monomers will be connected from head to tail
            link_distance (float): distance between consecutive monomers
        """
        #add start and end monomer gives flexibility on end groups
        self.start_monomer = start_monomer
        self.start = s_head
        self.end = s_tail
        self.monomer = monomer
        self.end_monomer = end_monomer
        self.n_units = n_units
        self.link_distance = link_distance
        #translate monomer so head atom is at the origin
        start_monomer.translate_sites(range(len(monomer)), - monomer.cart_coords[s_head])
        monomer.translate_sites(range(len(monomer)), - monomer.cart_coords[head])
        end_monomer.translate_sites(range(len(monomer)), - monomer.cart_coords[e_head])
        self.mon_vector = monomer.cart_coords[tail] - monomer.cart_coords[head]
        #add vector for terminal monomer
        self.end_mon_vector = end_monomer.cart_coords[e_tail] - end_monomer.cart_coords[e_head]
        self.moves = {1:[1,0,0], 2:[0,1,0], 3:[0,0,1], 4:[-1,0,0], 5:[0,-1,0], 6:[0,0,-1]}
        self.nmoves = len(self.moves.keys())
        self.prev_move = 1
        #places the start monomer at the beginning of the chain
        self.molecule = self.start_monomer.copy()
        self.length = 1
        self._create()
    
    def _create(self):
        """
        create the polymer from the monomer by random walk
        """
        #subtract 1 for the start monomer
        while self.length != (self.n_units - 1):
            monomer = self.monomer.copy()
            mon_vector = self.mon_vector
            self._add_monomer(monomer, mon_vector)
        
        end_monomer = self.end_monomer.copy()
        end_mon_vector = self.end_mon_vector
        self._add_terminal_monomer(end_monomer, end_mon_vector)
            
    def _next_move_direction(self):
        """
        pick a move at random from the list of moves
        """
        move = np.random.randint(1, self.nmoves+1)
        while self.prev_move == (move + 3)%6:
            move = np.random.randint(1, self.nmoves+1)
        self.prev_move = move
        print "move", move
        return np.array(self.moves[move])
    
    def _align_monomer(self, monomer, mon_vector, move_direction):
        """
        rotate the monomer so that it is aligned along the move direction
        """
        axis = np.cross(mon_vector, move_direction)
        origin = monomer[self.start].coords
        angle = get_angle(mon_vector, move_direction)
        #print "axis, origin, angle", axis, origin, angle
        op = SymmOp.from_origin_axis_angle(origin, axis, angle)
        monomer.apply_operation(op)

    def _add_monomer(self, monomer, mon_vector):
        """
        extend the polymer molecule by adding a monomer
        """
        #monomer = self.monomer.copy()
        move_direction = self._next_move_direction()
        translate_by = self.molecule.cart_coords[self.end] + self.link_distance * move_direction
        monomer.translate_sites(range(len(monomer)), translate_by)
        self._align_monomer(monomer, mon_vector, move_direction)
        # add monomer if there are no crossings
        does_cross = False
        for i, site in enumerate(monomer):
            try:
                self.molecule.append(site.specie, site.coords)
            except:
                does_cross = True
                polymer_length = len(self.molecule)
                self.molecule.remove_sites(range(polymer_length-i, polymer_length))
                break
        if not does_cross:
            self.length += 1
            self.end += len(self.monomer)
            print "length: ", self.length
            
    # this is unnecessary could just call _add_monomer in create???
    def _add_terminal_monomer(self, monomer, mon_vector):
        """
        add the last monomer unit
        """
        self._add_monomer(monomer, mon_vector)
        

In [3]:
peo_start = Molecule.from_file("PEOmonomer_start.xyz")
s_head = 0
s_tail = 5
peo_bulk = Molecule.from_file("PEOmonomer_bulk.xyz")
head = 0
tail = 4
peo_end = Molecule.from_file("PEOmonomer_end.xyz")
e_head = 0
e_tail = 4
n_units = 10
link_distance = 3.0

# create the polymer
peo_polymer = Polymer(peo_start, s_head, s_tail, peo_bulk, head, tail, peo_end, e_head, e_tail, n_units, link_distance)

move 6
length:  2
move 3
move 1
length:  3
move 1
length:  4
move 5
length:  5
move 3
length:  6
move 3
length:  7
move 3
length:  8
move 4
length:  9
move 4
length:  10


In [10]:
# write the polymer molecule
peo_polymer.molecule.to(filename="polymer.xyz", fmt="xyz")

Use openbabel to optimize the polymer chain.
Local optimization is used to preserve the chain structure.

In [5]:
from pymatgen.io.babel import BabelMolAdaptor

peo_obabel = BabelMolAdaptor(peo_polymer.molecule)
# ['gaff', 'ghemical', 'mmff94', 'mmff94s', 'uff']
peo_obabel.localopt(forcefield='mmff94', steps=500)
peo_optimized = peo_obabel.pymatgen_mol
peo_optimized.to(filename="polymer_optimized.xyz", fmt="xyz")

Get the polymer topology: atoms, bonds, angles and dihedrals

In [6]:
import openbabel

obmol = peo_obabel.openbabel_mol

atoms = [x.GetIdx()-1 for x in openbabel.OBMolAtomIter(obmol)]
bonds = [[x.GetBeginAtomIdx()-1, x.GetEndAtomIdx()-1] 
         for x in openbabel.OBMolBondIter(obmol)] 
angles = [list(x) for x in openbabel.OBMolAngleIter(obmol)]
dihedrals = [list(x) for x in openbabel.OBMolTorsionIter(obmol)]

print len(atoms), len(bonds), len(angles), len(dihedrals)

900 641 700 6
